In [8]:
import pandas as pd

In [9]:
ava_df = pd.read_csv('AVA.txt', sep=' ', names=['Image ID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Semantic ID 1', 'Semantic ID 2', 'Challenge ID'])
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
1,953619,0,1,5,17,38,36,15,6,5,1,1,22,1396
2,953958,10,7,15,26,26,21,10,8,1,2,1,21,1396
3,954184,0,0,4,8,41,56,10,3,4,0,0,0,1396
4,954113,0,1,4,6,48,37,23,5,2,2,15,21,1396
5,953980,0,3,6,15,57,39,6,1,1,1,22,38,1396


In [10]:
from skimage import io
import matplotlib.pyplot as plt
from tqdm import tqdm

In [11]:
ava_df['Mean Score'] = None

In [ ]:
import numpy as np
rows_to_delete = []
for i, row in tqdm(ava_df.iterrows()):
    try:
        _ = io.imread('images/{}.jpg'.format(row['Image ID']))
        prob_distr = row[[1,2,3,4,5,6,7,8,9,10]].values
        prob_distr = prob_distr / prob_distr.sum()
        weights = np.arange(10) + 1
        mean_score = (prob_distr * weights).sum()
        ava_df.loc[i, 'Mean Score'] = mean_score
    except:
        rows_to_delete.append(i)

17559it [11:56, 24.48it/s]

In [ ]:
proc_ava_df = ava_df.drop(rows_to_delete)
print('Removed {} rows'.format(len(rows_to_delete)))

In [ ]:
proc_ava_df.to_csv('proc_AVA.csv', index=False)